In [1]:
import pandas as pd
import numpy as np

In [ ]:
# 📌 1️⃣ Importación de librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from textwrap import wrap

# Descargar recursos de NLTK si es necesario
nltk.download('punkt')
nltk.download('stopwords')

# 📌 2️⃣ Carga y lectura del archivo TXT
file_path = "\Data\Banco-Preguntas-Icfes.txt"

with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Separar preguntas y respuestas
preguntas, respuestas = [], []
pregunta_actual, respuesta_actual = "", ""
leyendo_respuesta = False

for line in lines:
    if line.startswith("Pregunta:"):
        if pregunta_actual:
            preguntas.append(pregunta_actual.strip())
            respuestas.append(respuesta_actual.strip())
        pregunta_actual = line.replace("Pregunta:", "").strip()
        respuesta_actual = ""
        leyendo_respuesta = False
    elif line.startswith("Respuesta:"):
        leyendo_respuesta = True
        respuesta_actual = line.replace("Respuesta:", "").strip()
    else:
        if leyendo_respuesta:
            respuesta_actual += " " + line.strip()
        else:
            pregunta_actual += " " + line.strip()

# Añadir la última pregunta y respuesta
if pregunta_actual:
    preguntas.append(pregunta_actual.strip())
    respuestas.append(respuesta_actual.strip())

# Crear DataFrame
df = pd.DataFrame({"Pregunta": preguntas, "Respuesta": respuestas})

# 📌 3️⃣ Análisis estadístico básico
df["Num_Palabras_Pregunta"] = df["Pregunta"].apply(lambda x: len(word_tokenize(x)))
df["Num_Palabras_Respuesta"] = df["Respuesta"].apply(lambda x: len(word_tokenize(x)))

# Mostrar estadísticas generales
print("📊 Número total de preguntas:", len(df))
print("📊 Número total de respuestas:", len(df))
print("📊 Número total de palabras en preguntas:", df["Num_Palabras_Pregunta"].sum())
print("📊 Número total de palabras en respuestas:", df["Num_Palabras_Respuesta"].sum())

# 📌 4️⃣ Análisis de frecuencia de palabras
def limpiar_tokenizar(texto):
    tokens = word_tokenize(texto.lower())
    tokens = [re.sub(r'\W+', '', token) for token in tokens if token.isalpha()]
    return tokens

# Stopwords en español
stop_words = set(stopwords.words("spanish"))

# Tokenizar preguntas y respuestas
tokens_preguntas = [token for pregunta in df["Pregunta"] for token in limpiar_tokenizar(pregunta) if token not in stop_words]
tokens_respuestas = [token for respuesta in df["Respuesta"] for token in limpiar_tokenizar(respuesta) if token not in stop_words]

# Contar frecuencia de palabras
frecuencia_palabras_preguntas = Counter(tokens_preguntas)
frecuencia_palabras_respuestas = Counter(tokens_respuestas)

# Palabras más comunes
palabras_comunes_preguntas = frecuencia_palabras_preguntas.most_common(10)
palabras_comunes_respuestas = frecuencia_palabras_respuestas.most_common(10)

# 📌 5️⃣ Visualización de datos
plt.figure(figsize=(12, 6))

# Histograma para preguntas
plt.subplot(1, 2, 1)
sns.histplot(df["Num_Palabras_Pregunta"], bins=20, kde=True, color="blue")
plt.title("Distribución del Número de Palabras por Pregunta")
plt.xlabel("Número de Palabras")
plt.ylabel("Frecuencia")

# Histograma para respuestas
plt.subplot(1, 2, 2)
sns.histplot(df["Num_Palabras_Respuesta"], bins=20, kde=True, color="green")
plt.title("Distribución del Número de Palabras por Respuesta")
plt.xlabel("Número de Palabras")
plt.ylabel("Frecuencia")

plt.tight_layout()
plt.show()

# 📌 6️⃣ Generación de Nubes de Palabras
def generar_nube_palabras(data, titulo):
    wc = WordCloud(width=800, height=400, max_words=150, colormap="Dark2").generate_from_frequencies(data)
    plt.figure(figsize=(10, 6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title("\n".join(wrap(titulo, 60)), fontsize=13)
    plt.show()

# Nube de palabras para preguntas
generar_nube_palabras(frecuencia_palabras_preguntas, "Nube de Palabras - Preguntas")

# Nube de palabras para respuestas
generar_nube_palabras(frecuencia_palabras_respuestas, "Nube de Palabras - Respuestas")

# 📌 7️⃣ Palabras más comunes en preguntas y respuestas
palabras_preguntas, frecuencias_preguntas = zip(*palabras_comunes_preguntas)
palabras_respuestas, frecuencias_respuestas = zip(*palabras_comunes_respuestas)

plt.figure(figsize=(12, 6))

# Gráfico de barras para preguntas
plt.subplot(1, 2, 1)
sns.barplot(x=list(frecuencias_preguntas), y=list(palabras_preguntas), palette="viridis")
plt.title("Palabras Más Comunes en Preguntas")
plt.xlabel("Frecuencia")
plt.ylabel("Palabras")

# Gráfico de barras para respuestas
plt.subplot(1, 2, 2)
sns.barplot(x=list(frecuencias_respuestas), y=list(palabras_respuestas), palette="viridis")
plt.title("Palabras Más Comunes en Respuestas")
plt.xlabel("Frecuencia")
plt.ylabel("Palabras")

plt.tight_layout()
plt.show()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


FileNotFoundError: [Errno 2] No such file or directory: 'Banco-Preguntas-Icfes.txt'